# Zillow Real Estate Pricing (San Diego, CA)

## Functions and Libraries

In [1]:
import requests
import json
import csv
from bs4 import BeautifulSoup
import time

In [2]:
class ZillowScraper():
    
    results = []
    
    def __init__ (self, url, params, headers):
        self.url = url
        self.params = params
        self.headers = headers
        
    def pull(self):  
        response = requests.get(url=self.url, headers=self.headers, params=self.params)
        return response
     
        
    def analyze_html(self, response):
        
        content = BeautifulSoup(response,'lxml') 
        resident_cards = content.find('ul',{'class': 'photo-cards photo-cards_wow photo-cards_short'})
        for card in resident_cards.contents:
            script =  card.find('script', {'type':'application/ld+json'})
            
            if script:
                json_script = json.loads(script.contents[0])
                self.results.append({
                    'price': card.find('div',{'class':'list-card-price'}).text,
                    'postalcode': json_script['address']['postalCode'],
                    'city': json_script['address']['addressLocality'],
                    'url': json_script['url'],
                    'floorsize': json_script['floorSize']['value'],
                    'bedrooms': card.find_all('li')[0].text,
                    'bathrooms': card.find_all('li')[1].text,
                    'type': card.find('div',{'class':'list-card-type'}).text
                })
                
    
    def open_csv(self):
        with open('sandiego_zillow.csv','w') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=self.results[0].keys())
            writer.writeheader()
        
            for row in self.results:
                writer.writerow(row)
    
    def run(self,n):
               
        for page in range(1,n):
    
            res = self.pull()
            self.analyze_html(res.text)
            time.sleep(2)
        self.open_csv()   
        


## Scraping top San Diego suburbs

In [ ]:
san_diego_url = 'https://www.zillow.com/homes/San-Diego,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22San%20Diego%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.49774424785039%2C%22east%22%3A-116.54468028300664%2C%22south%22%3A32.66757437532881%2C%22north%22%3A33.01371776913924%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54296%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%7D%2C%22isListVisible%22%3Atrue%7D'
san_diego_params = {'searchQueryState': '{"pagination":{"currentPage": %s}:"San Diego, CA","mapBounds":{"west":-117.49774424785039,"east":-116.54468028300664,"south":32.66757437532881,"north":33.01371776913924},"regionSelection":[{"regionId":54296,"regionType":6}],"isMapVisible":true,"filterState":{},"isListVisible":true}'}
san_diego_headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                'accept-encoding': 'gzip, deflate, br',
                'accept-language': 'en-US,en;q=0.9',
                'cache-control': 'max-age=0',
                'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; zgsession=1|a6cb0293-619f-4881-b035-bbfb56f42169; KruxPixel=true; DoubleClickSession=true; KruxAddition=true; JSESSIONID=9D1E8FB0E97705D2BD01E93F00279956; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEmmXstMaGWiied6mz220J%2BKfkPIImmw0tPVmDqCrXF4O%2Bk%2BmHiLQarmwkTlbt%2Fui0wsqo0MT9rMM; _gid=GA1.2.392708527.1597802195; _derived_epik=dj0yJnU9Q20tZlFlcUdnVWk4Y2hZZG83NzFYYnlPTkZUUnY1Nm4mbj1aVkQxX29UeUNRYjQyREdIcVBad0hBJm09NyZ0PUFBQUFBRjg4aHRj; _px3=d443be8b33272214a9d0d5e9d83d16e9184c6b01165dd354031ee05e5b424eab:4K7NoT3k3hdkjxZpClPQzJinV+uy97j0wctfHLO7G4d5eGZBbanA7nLK5Fh5OdUNJ6yuR/wH+c+NWtzDiInqsw==:1000:bLCztu3syHOoORmqu0976t2LNac2WsCrFVCVCTuoR6DOw2g8nqywpzTvuyRpunh5DRJOq4b9mut9WZb5ulOVfMIdTQIY6whdyJzeer41CWbNEkrzOTFoOTAWdHqcd8XQkUctCNyRJcACoqnMb/IUok4jcMDa9MiQWvmRi+wDbqQ=; intercom-session-xby8p85u=L1dPZDl0dUdGblpmMnovb0hyaUFlSEVEZWRxUmdtc1kzdnV5Q0RyeGpSYWIrWXRxd0w0UlQ5WUVyVVl4MFY1ai0tMUpqcHpyMFdnRzhscmluYjVYN3hxQT09--e5209d9540b33f016a0488c9b2eb8d829e4361a1; _uetsid=4776572acf731b04a40223ea1fd6f89f; _uetvid=c2047787b1af49647f427b434721efa6; _gat=1; ki_t=1596088052742%3B1597763731703%3B1597802794834%3B4%3B111; AWSALB=oOJqzKC5UYOGg4HKEeomCFPM24FL4KJonHAoOzl2mtEq3XzZYya0+1zfWPTASYXM1vNihaDPkxMOrQVKGL9+c+5AxcDoOOCZj12xBsyPI6uumIS0TWjZvdOXnGxe; AWSALBCORS=oOJqzKC5UYOGg4HKEeomCFPM24FL4KJonHAoOzl2mtEq3XzZYya0+1zfWPTASYXM1vNihaDPkxMOrQVKGL9+c+5AxcDoOOCZj12xBsyPI6uumIS0TWjZvdOXnGxe; search=6|1600394794987%7Crect%3D33.01371776913924%252C-116.54468028300664%252C32.66757437532881%252C-117.49774424785039%26rid%3D54296%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26sort%3Ddays%26z%3D0%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0954296%09%09%09%09%09%09',
                'referer': 'https://www.zillow.com/homes/San-Diego,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22San%20Diego%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-118.04841296484373%2C%22east%22%3A-116.14228503515623%2C%22south%22%3A32.41986096386817%2C%22north%22%3A33.2276724215883%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54296%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A9%7D',
                'sec-fetch-dest': 'document',
                'sec-fetch-mode': 'navigate',
                'sec-fetch-site': 'same-origin',
                'sec-fetch-user': '?1',
                'upgrade-insecure-requests': '1',
                'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
              }

sandiego = ZillowScraper(san_diego_url, san_diego_params, san_diego_headers)
sandiego.run(21)

In [ ]:
chulavista_url = 'https://www.zillow.com/homes/Chula-Vista,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Chula%20Vista%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.14297535009766%2C%22east%22%3A-116.90608264990234%2C%22south%22%3A32.59346421848647%2C%22north%22%3A32.67008377843765%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A51405%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
chulavista_params = {
        'searchQueryState': '{"pagination":{},"usersSearchTerm":"Chula Vista, CA","mapBounds":{"west":-117.14297535009766,"east":-116.90608264990234,"south":32.59346421848647,"north":32.67008377843765},"regionSelection":[{"regionId":51405,"regionType":6}],"isMapVisible":true,"filterState":{"sch":{"value":true},"sort":{"value":"globalrelevanceex"}},"isListVisible":true,"mapZoom":12}'
                    }
chulavista_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; _gid=GA1.2.422558928.1598197444; DoubleClickSession=true; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEmoVDil444xOoWriMVHhRnmZtTup%2BiaqpD90UpoS9nJtbLajmis2YvScdJMzJN%2FK2zAf8oFDCXMo; JSESSIONID=3CFED4545389AE56DDB19176989B1816; KruxPixel=true; KruxAddition=true; _derived_epik=dj0yJnU9V21iaEJ5ZkxBMFBFY3FfTzE1c1ozY2xIZkdrX19xeEQmbj1nSlJuZm5fVUlEWFdlcmM5anB2bXFnJm09NyZ0PUFBQUFBRjlDNktB; _gat=1; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; _pxff_bsco=1; _px3=bdc7057e1f4f4687f992dba92a751634de7b914c02cb11554cf779524f53c915:JuC1ucon+WaUD6EnLaKGnRonxaHeNBOivYGbypqwZ/XPisZXPvw3AuUKD8Bs5UoeTRsop9oS5aQ4krnQ7Gh4gQ==:1000:Qx1QMh0Psym9qXowbpPlv0S87LUi7PVGWzmEoE5XNNjAd/4UybEMj4wdsEoLGxXoGf6V2QgxnXGA7i0gb6bcm3vyZ8I42p8VYCB3Yj1RzlO6yFyGwi7aYrwsoftvpoTGc3VXS+0UAhUbDsYa5omcHoRvygNErca2aOGgrvVog+w=; intercom-session-xby8p85u=Wk00MEhhaDBBRTcxbVpjTERJajVOeVg4amE4NnlpVnBYYVE2UUtFK1FaYlV6cW1SemZsT09oNVduYnppZE1RZi0teEgvdWNlV2F1VnlObUNtRWlZNUtVUT09--31eeeafafaede1a570ef927b08649ea62d7f64e5; _uetsid=12165f4709e6d8c837d063c0e3be9cca; _uetvid=c2047787b1af49647f427b434721efa6; AWSALB=FhDNcWc8hxtHDsdqb4jIfUZZKKNKhT9Uvu9UlaYDh3WaNj3Okeh7hDz7T5d8QGVZfH81SaPBv3KQSTpD71QTGfxCUQHu7exarDymES9u9VzrshS1wrjiDNDPIWhX; AWSALBCORS=FhDNcWc8hxtHDsdqb4jIfUZZKKNKhT9Uvu9UlaYDh3WaNj3Okeh7hDz7T5d8QGVZfH81SaPBv3KQSTpD71QTGfxCUQHu7exarDymES9u9VzrshS1wrjiDNDPIWhX; search=6|1600812635107%7Crect%3D32.67008377843765%252C-116.90608264990234%252C32.59346421848647%252C-117.14297535009766%26rid%3D51405%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D0%26sch%3Dprcemh1u%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0951405%09%09%09%09%09%09',
        'referer': 'https://www.zillow.com/homes/Chula-Vista,-CA_rb/',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
}

chulavista = ZillowScraper(chulavista_url, chulavista_params, chulavista_headers)
chulavista.run(9)

In [ ]:
lamesa_url = 'https://www.zillow.com/homes/La-Mesa,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22La%20Mesa%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.11242935009767%2C%22east%22%3A-116.87553664990236%2C%22south%22%3A32.72962949424513%2C%22north%22%3A32.806132295074505%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A46089%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
lamesa_params = {'searchQueryState': '{"pagination":{},"usersSearchTerm":"La Mesa, CA","mapBounds":{"west":-117.11242935009767,"east":-116.87553664990236,"south":32.72962949424513,"north":32.806132295074505},"regionSelection":[{"regionId":46089,"regionType":6}],"isMapVisible":true,"filterState":{"sch":{"value":true},"sort":{"value":"globalrelevanceex"}},"isListVisible":true,"mapZoom":12}'}
lamesa_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; _gid=GA1.2.422558928.1598197444; DoubleClickSession=true; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEmoVDil444xOoWriMVHhRnmZtTup%2BiaqpD90UpoS9nJtbLajmis2YvScdJMzJN%2FK2zAf8oFDCXMo; JSESSIONID=3CFED4545389AE56DDB19176989B1816; KruxPixel=true; KruxAddition=true; _gat=1; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; _pxff_bsco=1; _derived_epik=dj0yJnU9cFZpbWtYMjJUQmZIRlF6TURpb2pzTU4zUEVRTElONUYmbj1YZWFlX2RYVnFVN3FOa0Q0ZWZvTjRRJm09NyZ0PUFBQUFBRjlDNzlR; _px3=dd510856a30ffe4f2f906f2fe8c9e3de0aec946a88e3685b69f027515cee9335:8hrxMCfdeyls5vfcPxFiUo8ePoHe15yGQIrrGKmwqliWlPlFb8VDE3dKj322LnFqIPuTKYk0YUHvaZbpesH5xg==:1000:fY4RBvP1jUCkF+XW0mfIvZx6NdIP8Mq35VV6Whrku9wu/5ypH2ln37Q+AeDis+MyPDm4ycw2YNwycO1F9yCxVrDNljCR+WnLcFuE6P2a+z/2JR8po6jKgm5yRMqpnpmcDuO0qT8QXnS8CTjNk/Z64lo05XeWmTukrq+rNIODwr8=; intercom-session-xby8p85u=MGJWdTU0TDJlR2N0ZGdGUzZYTHZxcWxhQVhsM1dxRU4ydDJRemN0eHJ1TTR1MHQrUGRURSszUnNHK05rT2pITi0tMzdKVWUwMFNtbThHZGJwRVpJSDdmdz09--d13189df1f2b9ae11f5bc50a186619727fc64250; _uetsid=12165f4709e6d8c837d063c0e3be9cca; _uetvid=c2047787b1af49647f427b434721efa6; AWSALB=Ex/ROIJL7p2MJA6DBQAv9MZbqa9rLoBh470ey4mGy53kTkGQUMmCZDFNkopttYN4J2T5aySBpgAAhkJtSJxpCxThoSYhZHRo5QNLWxsZMW8MJwUiBXpVnG2umxCb; AWSALBCORS=Ex/ROIJL7p2MJA6DBQAv9MZbqa9rLoBh470ey4mGy53kTkGQUMmCZDFNkopttYN4J2T5aySBpgAAhkJtSJxpCxThoSYhZHRo5QNLWxsZMW8MJwUiBXpVnG2umxCb; search=6|1600814302472%7Crect%3D32.806132295074505%252C-116.87553664990236%252C32.72962949424513%252C-117.11242935009767%26rid%3D46089%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D0%26sch%3Dprcemh1u%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0946089%09%09%09%09%09%09',
    'referer': 'https://www.zillow.com/homes/Escondido,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Escondido%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.51274940039063%2C%22east%22%3A-116.56517859960938%2C%22south%22%3A32.99677584858403%2C%22north%22%3A33.358839537831976%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A11337%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%7D',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
}

lamesa = ZillowScraper(lamesa_url,lamesa_params,lamesa_headers)
lamesa.run(4)

In [ ]:
elcajon_url = 'https://www.zillow.com/homes/El-Cajon,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22El%20Cajon%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.01434835009763%2C%22east%22%3A-116.77745564990232%2C%22south%22%3A32.77151445525491%2C%22north%22%3A32.84798125371526%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A38333%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
elcajon_params = {
    'searchQueryState': '{"pagination":{},"usersSearchTerm":"El Cajon, CA","mapBounds":{"west":-117.01434835009763,"east":-116.77745564990232,"south":32.77151445525491,"north":32.84798125371526},"regionSelection":[{"regionId":38333,"regionType":6}],"isMapVisible":true,"filterState":{"sch":{"value":true},"sort":{"value":"globalrelevanceex"}},"isListVisible":true,"mapZoom":12}'
}
elcajon_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; _gid=GA1.2.422558928.1598197444; DoubleClickSession=true; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEmoVDil444xOoWriMVHhRnmZtTup%2BiaqpD90UpoS9nJtbLajmis2YvScdJMzJN%2FK2zAf8oFDCXMo; JSESSIONID=3CFED4545389AE56DDB19176989B1816; KruxPixel=true; KruxAddition=true; _derived_epik=dj0yJnU9YjVRMTRpb0JSYlpxQjNpUVE5Qm9zS1A4SGZoSGlpZGgmbj1lbTktcEFQUFBYZkRZSjAwZENoTXZBJm09NyZ0PUFBQUFBRjlDLWZJ; _gat=1; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; _pxff_bsco=1; _px3=3d2421128708e90c4ad312b5bc7c1b430bc50d3cf07ee06e0305d089b4d447f5:ZLC14YV8SC+anaO1YRAskCWn9JNTz3HLEh5X4yBkmb4O2cI3qASRUElUzpyb5JbHf+TJAPi4+ramu800XOXTPQ==:1000:KvlW71KACBYgdYtIE8tDoZFu+mNqJEx3Q5tUE+QNYfj36FUPZwAkNLF7+xyJTClh9QMbL/dVSg27V2MlKMuCjKrNegP7VguZR7plrXkO+llgF05j2xeMoLRqGQYchhXntMf5MmlDtu1paOuRKXJlL4v5UyJ9VfS7HMH1KqE34U4=; intercom-session-xby8p85u=WkhIQk9CTzdYMndpdUJ0bUsyd1dPa2ZlTDBESUpVVFBhT0xpMlRtaDhidzM4QnBxUnpPVVIrVGpPNkZ4bGFFZC0tMUdaaVZGMzZhM0FBb0R3b3N3Q2ludz09--1564ad215ec7376d84aba1febf67d680b596a97a; _uetsid=12165f4709e6d8c837d063c0e3be9cca; _uetvid=c2047787b1af49647f427b434721efa6; AWSALB=HHbLgsrwuunD9m5iLeeV6Z1ybcFWLtOY/Kbt0d7Uc3iSq4FN+Km6u/jkweWYdehrmbt2AD26VjNvQE5qTkRWoOyg/vP++H0o4VbWcM3jSsxcU+H/UhNuEVcA0vYs; AWSALBCORS=HHbLgsrwuunD9m5iLeeV6Z1ybcFWLtOY/Kbt0d7Uc3iSq4FN+Km6u/jkweWYdehrmbt2AD26VjNvQE5qTkRWoOyg/vP++H0o4VbWcM3jSsxcU+H/UhNuEVcA0vYs; search=6|1600817011174%7Crect%3D32.84798125371526%252C-116.77745564990232%252C32.77151445525491%252C-117.01434835009763%26rid%3D38333%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D0%26sch%3Dprcemh1u%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0938333%09%09%09%09%09%09',
    'referer': 'https://www.zillow.com/homes/El-Cajon,-CA_rb/',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
}

elcajon = ZillowScraper(elcajon_url, elcajon_params,elcajon_headers)
elcajon.run(8)

In [ ]:
carlsbad_url = 'https://www.zillow.com/homes/Carlsbad,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Carlsbad%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.39995085009765%2C%22east%22%3A-117.16305814990234%2C%22south%22%3A33.08372161714682%2C%22north%22%3A33.159918772259466%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A3975%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
carlsbad_params = {
    'searchQueryState': '{"pagination":{},"usersSearchTerm":"Carlsbad, CA","mapBounds":{"west":-117.39995085009765,"east":-117.16305814990234,"south":33.08372161714682,"north":33.159918772259466},"regionSelection":[{"regionId":3975,"regionType":6}],"isMapVisible":true,"filterState":{"sch":{"value":true},"manu":{"value":false},"land":{"value":false}},"isListVisible":true,"mapZoom":12}'
}
carlsbad_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; DoubleClickSession=true; KruxPixel=true; KruxAddition=true; _gid=GA1.2.612872810.1598407954; JSESSIONID=89B940B5E855310E6E0F52F0B99622DB; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEtTcb3iksrkhx1yTFnFnp8o2IZp4XX0ywk1a4aJenH7ifsHeorNmdZcF4oRi2wxG3eH2OEFmUYmr; _derived_epik=dj0yJnU9MDMtdFlJV2haWE11YUM3emFTMV9KOHo4dUtDQjZORk8mbj1DOEo0TjQ4ZWsyUmRZZnprbjNELTJBJm09NyZ0PUFBQUFBRjlGOXFJ; _gat=1; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; _pxff_bsco=1; _px3=45163a2f6217520769e17355705f65072a3adac621f6e906671fb1eb128236da:yUaqp95JauJATd35c5U8I0mACyuiD6X4COR9n7lmhfl/utfIlvsh9skudqjIgB+U+GmXlhe31RIYUfKAdvwgbw==:1000:Y4ZwpBfqScw9B2vSyNgi8bsOSgbke2mPaJBYeiJvPKgXStAfweY3ORM3f9rHmKyOhygRzL5/JXV56vMOdBM9YsSvXpaYsKZfRID4qPW7L4RpMKmGMTDEBTHzDZd+HfQ6+TwqQbSIJslCMpsGFzaT3agByvFky/Z4Nnv/LRaAiyU=; intercom-session-xby8p85u=cDdFeis3WnAvdDhObzdWeFBTbVE0YXFySENqMHJNSGZSL0R5dVBnbTBXaHkxdjdrY3R2eENrK2FrTzhPNGtVRS0tQW5DV2RlekQydFBiYksxTitCbkw2QT09--5ebd9c1ec18c532a9685f9ebb98dd95af2adf613; _uetsid=0d2e531968d37437d5cce675ebb2eb1c; _uetvid=c2047787b1af49647f427b434721efa6; AWSALB=UWz20lxLyFSlRRblRFEPVEtSQEyp9ZwiAzRcbPDksemjPxTaSKRP3mIJrBYYPSAJbq5Sg1BP7JFjUsWhAg1rmH6hcFVW4KiQz9r2F7fDnh9qwWUXVujXrmIz3adG; AWSALBCORS=UWz20lxLyFSlRRblRFEPVEtSQEyp9ZwiAzRcbPDksemjPxTaSKRP3mIJrBYYPSAJbq5Sg1BP7JFjUsWhAg1rmH6hcFVW4KiQz9r2F7fDnh9qwWUXVujXrmIz3adG; search=6|1601013907419%7Crect%3D33.159918772259466%252C-117.16305814990234%252C33.08372161714682%252C-117.39995085009765%26rid%3D3975%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26sort%3Ddays%26z%3D0%26sch%3Dprcemh1u%26type%3Dhouse%252Ccondo%252Capartment_duplex%252Ctownhouse%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%093975%09%09%09%09%09%09',
    'referer': 'https://www.zillow.com/homes/Carlsbad,-CA_rb/',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
}

carlsbad = ZillowScraper(carlsbad_url,carlsbad_params,carlsbad_headers)
carlsbad.run(7)

In [ ]:
escondido_url = 'https://www.zillow.com/homes/Escondido,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Escondido%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.27585670019532%2C%22east%22%3A-116.8020712998047%2C%22south%22%3A33.10181322478786%2C%22north%22%3A33.25410997538318%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A11337%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D'
escondido_params = {
    'searchQueryState': '{"pagination":{},"usersSearchTerm":"Escondido, CA","mapBounds":{"west":-117.51274940039063,"east":-116.56517859960938,"south":33.0255656707587,"north":33.33015901794086},"regionSelection":[{"regionId":11337,"regionType":6}],"isMapVisible":true,"filterState":{"sch":{"value":true},"sort":{"value":"globalrelevanceex"}},"isListVisible":true}'
} 
escondido_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; _gid=GA1.2.422558928.1598197444; DoubleClickSession=true; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEmoVDil444xOoWriMVHhRnmZtTup%2BiaqpD90UpoS9nJtbLajmis2YvScdJMzJN%2FK2zAf8oFDCXMo; JSESSIONID=3CFED4545389AE56DDB19176989B1816; KruxPixel=true; KruxAddition=true; _derived_epik=dj0yJnU9V21iaEJ5ZkxBMFBFY3FfTzE1c1ozY2xIZkdrX19xeEQmbj1nSlJuZm5fVUlEWFdlcmM5anB2bXFnJm09NyZ0PUFBQUFBRjlDNktB; _pxff_tm=1; _px3=dcd75056dbf9f561e239b35cd1220160fc282537c47cfc8b2f1af1775c591b6e:+4TvqdKBusabF4mlWbIZgxjgOmhUMzAUVgbXKk9NQC+xl/UbpszhqPfcVvkj4Uy8DXEnOHiS83orpPkgKVQl2w==:1000:pJp+UY3z5FoAocoC1nq0lvw1anuSi7dKoX6v9f44R42OKWAve0sIXqWbQZBCzZUlOAbOe6tqTSnQgjoB1aBwA9qUJ9B9hCr0T7Xr8Z394P7y2BD8yhJjvI25R0CM6uRwcgUWccAhHiq3/kGJqcb00bqKwpIgHPHS5Vi63T8iLHg=; _uetsid=12165f4709e6d8c837d063c0e3be9cca; _uetvid=c2047787b1af49647f427b434721efa6; intercom-session-xby8p85u=UjV3TkNNYi9XWTZEdGl4RTkvcWZHM0I5T2s1Y1dJc0lxNEVNNHJneUk2Q2tidTJzclRTZTBDckFnOXUraEx2Qy0tQ3JjbGxLWmF2andDc1JWZlY5SEVmQT09--f1af0facef2b02fe8efc8bfd3b9448e57375b7ec; AWSALB=EUBOhfIXVhnrLpZkrZZHdOOYhjjyZnGmU0Xwl66CQOX5vA54p/Xs1TqDsFt91zEZn77ObMf1/+IoR5gxa3D6g8+ux8d/sYaa2HYeKs80o65Fa/48pLKmNxQo/4/N; AWSALBCORS=EUBOhfIXVhnrLpZkrZZHdOOYhjjyZnGmU0Xwl66CQOX5vA54p/Xs1TqDsFt91zEZn77ObMf1/+IoR5gxa3D6g8+ux8d/sYaa2HYeKs80o65Fa/48pLKmNxQo/4/N; search=6|1600813122433%7Crect%3D33.33015901794086%252C-116.56517859960938%252C33.0255656707587%252C-117.51274940039063%26rid%3D11337%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D0%26sch%3Dprcemh1u%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0911337%09%09%09%09%09%09; _gat=1',
    'referer': 'https://www.zillow.com/homes/Escondido,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Escondido%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.51274940039063%2C%22east%22%3A-116.56517859960938%2C%22south%22%3A32.81460699597056%2C%22north%22%3A33.53988170159496%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A11337%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%7D',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
}

escondido = ZillowScraper(escondido_url,escondido_params,escondido_headers)
escondido.run(7)

In [ ]:
oceanside_url = 'https://www.zillow.com/homes/Oceanside,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Oceanside%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-118.36493880078126%2C%22east%22%3A-116.46979719921876%2C%22south%22%3A33.023607227575525%2C%22north%22%3A33.63174637667549%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A6285%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A9%7D'
oceanside_params = {'searchQueryState': '{"pagination":{},"usersSearchTerm":"Oceanside, CA","mapBounds":{"west":-117.65426070019532,"east":-117.1804752998047,"south":33.25215662572245,"north":33.4041917927507},"regionSelection":[{"regionId":6285,"regionType":6}],"isMapVisible":true,"filterState":{"sch":{"value":true},"sort":{"value":"globalrelevanceex"}},"isListVisible":true,"mapZoom":11}'}
oceanside_headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                    'accept-encoding': 'gzip, deflate, br',
                    'accept-language': 'en-US,en;q=0.9',
                    'cache-control': 'max-age=0',
                    'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; _gid=GA1.2.422558928.1598197444; DoubleClickSession=true; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEmoVDil444xOoWriMVHhRnmZtTup%2BiaqpD90UpoS9nJtbLajmis2YvScdJMzJN%2FK2zAf8oFDCXMo; JSESSIONID=3CFED4545389AE56DDB19176989B1816; KruxPixel=true; _derived_epik=dj0yJnU9UWxZOXFYNm4zNkJ4UE95T28xQXFVeXlSdlNpdWstTm8mbj1fRmVpakRjSUo0bUc5dlR0SlcwRFNnJm09NyZ0PUFBQUFBRjlDM21F; KruxAddition=true; _gat=1; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; _pxff_bsco=1; _px3=6de23c6f4b310ad654679ffc11f3130ccc4b8aff3664a96b8b574a408a2871df:2AaLwE+EOJEo5Efa9iV0Q39GT2a2PZf/G3qcDyPsvR7y8a5/uCpIL1PsaLtyOvXwhAlwMWH23ynvQGCipZvnNA==:1000:e2D4rk6dvINd9vzwA5cVwTIgUcFDvatjAZO42QxrDUyBPhcsDlLSn2tQShPqIQBbx0d4/ccbpFgALCFymTso2qpbe4mme8fFq+jdErZ42vy86suFZbU+r+Ksvy6DKHDk0LLCM/6S3O7GepunTrPzP6C6rB8xdvv7tbcWmJDthoI=; intercom-session-xby8p85u=dHZBS3cwUU9iR0lTOEl4dVFPTjY1Sld2VUZYNzcyVXY0YThoTWRHMmdQVHNqSU95VHRQVmFxZTV0OWlIYTFhTy0tUVlOTHN2OERrM2RQMSthWnY3eWRTdz09--c085ca368f0cf1c92cd0fb78e7c54cab735ca39c; _uetsid=12165f4709e6d8c837d063c0e3be9cca; _uetvid=c2047787b1af49647f427b434721efa6; AWSALB=9Jiy6UtmIc9YdBXs1K/SHNXgbXYIBGhGnaz/LghsGpnRVpo0VZqLJZJGJGTP18U0lFha3bLxBKcZBDgv6Y9/yqIjBwly7gVucNIREvKr5QG6DE+s0DTHfBHx98wB; AWSALBCORS=9Jiy6UtmIc9YdBXs1K/SHNXgbXYIBGhGnaz/LghsGpnRVpo0VZqLJZJGJGTP18U0lFha3bLxBKcZBDgv6Y9/yqIjBwly7gVucNIREvKr5QG6DE+s0DTHfBHx98wB; search=6|1600811038773%7Crect%3D33.4041917927507%252C-117.1804752998047%252C33.25215662572245%252C-117.65426070019532%26rid%3D6285%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D0%26sch%3Dprcemh1u%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%096285%09%09%09%09%09%09',
                    'referer': 'https://www.zillow.com/homes/Oceanside,-CA_rb/',
                    'sec-fetch-dest': 'document',
                    'sec-fetch-mode': 'navigate',
                    'sec-fetch-site': 'same-origin',
                    'sec-fetch-user': '?1',
                    'upgrade-insecure-requests': '1',
                    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
                    }

oceanside = ZillowScraper(oceanside_url,oceanside_params,oceanside_headers)
oceanside.run(9)

In [ ]:
lajolla_url = 'https://www.zillow.com/homes/la-jolla,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22la%20jolla%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.31402417504884%2C%22east%22%3A-117.19557782495119%2C%22south%22%3A32.82885020178775%2C%22north%22%3A32.867067157909574%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A46087%2C%22regionType%22%3A8%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A13%7D'
lajolla_params = {
    'searchQueryState': '{"pagination":{},"usersSearchTerm":"la jolla, CA","mapBounds":{"west":-117.31402417504884,"east":-117.19557782495119,"south":32.82885020178775,"north":32.867067157909574},"regionSelection":[{"regionId":46087,"regionType":8}],"isMapVisible":true,"filterState":{"sch":{"value":true},"sort":{"value":"globalrelevanceex"},"manu":{"value":false},"land":{"value":false}},"isListVisible":true,"mapZoom":13}'
}
lajolla_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; _gid=GA1.2.422558928.1598197444; DoubleClickSession=true; KruxPixel=true; KruxAddition=true; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEqAJX2RdwVo6bgahT3PUL4CD4yApNEmKum1WrkrJMrOxq3DheYoJkPkRX4rFeZ2kSfuwn3aVerE5; _gat=1; JSESSIONID=BD9EF2A91C38DF63A6B3755D0FEACB0D; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; _pxff_bsco=1; _derived_epik=dj0yJnU9LThxMkZmTm01RFhtQVpOOGhMbWhZV203cVIya0FtYy0mbj0tTFowUmplU283a2JvaXRheUp0Q1p3Jm09NyZ0PUFBQUFBRjlESnZj; _px3=c0994670efadfe3da35a3e101edc45e101d7da4dae07415bf958b0d20ed92520:Dah3e1RP3mmQtUXNu5yvg1+UQ+QzGBTb+1+VpTfRM8DKFHUOoxwCIq/O/vz43YoMlyzyk1X9OPTTuywCD2Ki9A==:1000:Y/uTux4+ePgNKLkoSfNflNhojuom9eg9Sv4EY59oLm5zgxyqX93lLTRLen/6/emwB4myWFgNkTrL24qurhbQlOwOMsixvmRUhIN6/04OQ6rjJjxBqvGcSx+RyuTyt/o2m8PUbO/PKcQIPER4vJNwGpQdBvVhiCFIJy1OW8Vq6xQ=; intercom-session-xby8p85u=cmZsN2pZR1JuS0RXS1VoNzlJYjVsTjIxYS9LbTZYNWJETDY3T09SVDlUZ3FtamoxcXg3RTUzTWtJZkJBbC9IWi0tc3ZVa2VwS2Mwc2JnNUVTZ3o2TDBxQT09--851ac557faad20a1a53ec4bf8011cff2e35a8dc8; _uetsid=12165f4709e6d8c837d063c0e3be9cca; _uetvid=c2047787b1af49647f427b434721efa6; AWSALB=QUl/1FXi7T/iEOGHXLalqs/ARLXC/c8UNgupDkIAEhoNkDqCj+XuXNG/AWerHZFUI0kOGm4+aPvhVksSDwGGX1VlWuJO8E5pM4onYGKXETM7Ncp69jlgAQRO7jK7; AWSALBCORS=QUl/1FXi7T/iEOGHXLalqs/ARLXC/c8UNgupDkIAEhoNkDqCj+XuXNG/AWerHZFUI0kOGm4+aPvhVksSDwGGX1VlWuJO8E5pM4onYGKXETM7Ncp69jlgAQRO7jK7; search=6|1600828432098%7Crect%3D32.867067157909574%252C-117.19557782495119%252C32.82885020178775%252C-117.31402417504884%26rid%3D46087%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D0%26sch%3Dprcemh1u%26type%3Dhouse%252Ccondo%252Capartment_duplex%252Ctownhouse%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0946087%09%09%09%09%09%09',
    'referer': 'https://www.zillow.com/homes/la-jolla,-CA_rb/',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
}

lajolla = ZillowScraper(lajolla_url, lajolla_params, lajolla_headers)
lajolla.run(8)

In [ ]:
encinitas_url = 'https://www.zillow.com/homes/Encinitas,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Encinitas%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.37194085009766%2C%22east%22%3A-117.13504814990235%2C%22south%22%3A33.01150362015394%2C%22north%22%3A33.08776334881521%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A45164%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
encinitas_params = {
    'searchQueryState': '{"pagination":{},"usersSearchTerm":"Encinitas, CA","mapBounds":{"west":-117.37194085009766,"east":-117.13504814990235,"south":33.01150362015394,"north":33.08776334881521},"regionSelection":[{"regionId":45164,"regionType":6}],"isMapVisible":true,"filterState":{"sch":{"value":true},"manu":{"value":false},"land":{"value":false}},"isListVisible":true,"mapZoom":12}'
}
encinitas_headers = {
                    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                    'accept-encoding': 'gzip, deflate, br',
                    'accept-language': 'en-US,en;q=0.9',
                    'cache-control': 'max-age=0',
                    'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; DoubleClickSession=true; KruxPixel=true; KruxAddition=true; _gid=GA1.2.612872810.1598407954; JSESSIONID=89B940B5E855310E6E0F52F0B99622DB; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEtTcb3iksrkhx1yTFnFnp8o2IZp4XX0ywk1a4aJenH7ifsHeorNmdZcF4oRi2wxG3eH2OEFmUYmr; _derived_epik=dj0yJnU9MDMtdFlJV2haWE11YUM3emFTMV9KOHo4dUtDQjZORk8mbj1DOEo0TjQ4ZWsyUmRZZnprbjNELTJBJm09NyZ0PUFBQUFBRjlGOXFJ; _gat=1; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; _pxff_bsco=1; _px3=630bf8c71b4898e81db20432e92eae84a473e38156f4a2bc518d3b349c306a7a:UmgL6SdkDzGH3TLKiS9uANxZu6TrSZinj4id9IYplsgAgFeOsLkqlyM79Pd2wV3EVr1XCXJ7B+BYdnNZThmhsA==:1000:0URPz0JeKaD7gg8p6uzQHbbjbupyTlpdHwfiHJI0CYX8XXMEwv6nYYiY8lr8AhJ9FyIHlt8YJ1Bo3l7neSJzNRqritr65hIfprwTUsWLLypSYmvTph8IaVylA98RaT7ceFJ62ssHW9XqYBc3XCPDMXLzfyp6PuHXESc8xzsf3O4=; intercom-session-xby8p85u=WmsvVDFXNkhPVXJLVm9FVTF3RjJONUhsc3NqeEYvN1ZVSUFHQ2UyK1NXYm1jbGVqemYvU3A0Rkc3MmUwdk9hVi0tbFhIYXRjWVVpQmcvOVNvV0E0SWk1dz09--f823c7997dba353486cf0a29637a7605309860b9; _uetsid=0d2e531968d37437d5cce675ebb2eb1c; _uetvid=c2047787b1af49647f427b434721efa6; AWSALB=4vsICRcAJmnZNOoNymP816jjVxJ+gYsrgGRyDE2eo2P6vXyAls9Lk/5ccZiPrSmHvNsdLWf67Rq17Q7XJv9oNiK2dYoUhcrJVKrVkgfV2WDjomuUyZhLnmjsvS6S; AWSALBCORS=4vsICRcAJmnZNOoNymP816jjVxJ+gYsrgGRyDE2eo2P6vXyAls9Lk/5ccZiPrSmHvNsdLWf67Rq17Q7XJv9oNiK2dYoUhcrJVKrVkgfV2WDjomuUyZhLnmjsvS6S; search=6|1601013598717%7Crect%3D33.08776334881521%252C-117.13504814990235%252C33.01150362015394%252C-117.37194085009766%26rid%3D45164%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26sort%3Ddays%26z%3D0%26sch%3Dprcemh1u%26type%3Dhouse%252Ccondo%252Capartment_duplex%252Ctownhouse%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0945164%09%09%09%09%09%09',
                    'referer': 'https://www.zillow.com/homes/Encinitas,-CA_rb/',
                    'sec-fetch-dest': 'document',
                    'sec-fetch-mode': 'navigate',
                    'sec-fetch-site': 'same-origin',
                    'sec-fetch-user': '?1',
                    'upgrade-insecure-requests': '1',
                    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
}

encinitas = ZillowScraper(encinitas_url, encinitas_params, encinitas_headers)
encinitas.run(5)

In [ ]:
sanmarcos_url = 'https://www.zillow.com/homes/San-Marcos,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22San%20Marcos%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.28933135009765%2C%22east%22%3A-117.05243864990234%2C%22south%22%3A33.1214026985497%2C%22north%22%3A33.197567156747546%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A40737%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
sanmarcos_params = {
    'searchQueryState': '{"pagination":{},"usersSearchTerm":"San Marcos, CA","mapBounds":{"west":-117.28933135009765,"east":-117.05243864990234,"south":33.1214026985497,"north":33.197567156747546},"regionSelection":[{"regionId":40737,"regionType":6}],"isMapVisible":true,"filterState":{"sch":{"value":true},"manu":{"value":false},"land":{"value":false}},"isListVisible":true,"mapZoom":12}'
                    }
sanmarcos_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; DoubleClickSession=true; KruxPixel=true; KruxAddition=true; _gid=GA1.2.612872810.1598407954; JSESSIONID=89B940B5E855310E6E0F52F0B99622DB; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEtTcb3iksrkhx1yTFnFnp8o2IZp4XX0ywk1a4aJenH7ifsHeorNmdZcF4oRi2wxG3eH2OEFmUYmr; _derived_epik=dj0yJnU9MDMtdFlJV2haWE11YUM3emFTMV9KOHo4dUtDQjZORk8mbj1DOEo0TjQ4ZWsyUmRZZnprbjNELTJBJm09NyZ0PUFBQUFBRjlGOXFJ; _gat=1; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; _pxff_bsco=1; _px3=7170651eb13a862800f3ad579a0d37e10b3072b6a4f04d0f5b46fe3fbcec638a:l70r51Hu2PrGPJEZxqiSvm1hDBCQN3+DRTkOjNwVWSkUE79U5j6nqmOFcx12Ee8zJjFfcEXrwFsgVZ3uzL1ugQ==:1000:UzrqbkWHKFTmT7JEkofsDOxufWL5MhAMehRQh7GcUfVmAmKBQDaYAjUb8tDfErOO/GlHr3KZ89IvhGoOz26HZxWnWMeZYxjXL9gDiIzT7KMy7UYNaORgbA7mjMLobkQk2jxzmTfqBZVv0GwmfbEdAISHWQTMJ8NBeOH/jDCXT/E=; intercom-session-xby8p85u=NFdjTDBBTnFUcmRoZjdicGRNOE43ZHpRWHduaXJESmF4YzJIMUo5QnVLMFYwN1pkdi9tZXoxMFp5OEs5MWhhZi0tWVVxMlBDbXFQVXdMTlMxd2JJRG5CUT09--4c34463d136bace70432f430db5fe624944bc736; _uetsid=0d2e531968d37437d5cce675ebb2eb1c; _uetvid=c2047787b1af49647f427b434721efa6; AWSALB=NT4yVDr1qXAt8KamFA8HSc3wuZRYgYOLRVQz8AT1KutY3yohq4KU3Pfu7vySiTG8Qhoqb34SmEjcTBlc10ixZU3W5a9asY4QoXAhH9ZWUdr017sxII21LjoMh63b; AWSALBCORS=NT4yVDr1qXAt8KamFA8HSc3wuZRYgYOLRVQz8AT1KutY3yohq4KU3Pfu7vySiTG8Qhoqb34SmEjcTBlc10ixZU3W5a9asY4QoXAhH9ZWUdr017sxII21LjoMh63b; search=6|1601014251765%7Crect%3D33.197567156747546%252C-117.05243864990234%252C33.1214026985497%252C-117.28933135009765%26rid%3D40737%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26sort%3Ddays%26z%3D0%26sch%3Dprcemh1u%26type%3Dhouse%252Ccondo%252Capartment_duplex%252Ctownhouse%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0940737%09%09%09%09%09%09',
    'referer': 'https://www.zillow.com/homes/San-Marcos,-CA_rb/',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
                    }

sanmarcos = ZillowScraper(sanmarcos_url,sanmarcos_params,sanmarcos_headers)
sanmarcos.run(4)


In [ ]:
solanabeach_url = 'https://www.zillow.com/homes/Solana-Beach,-CA_rb/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Solana%20Beach%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-117.28874408752442%2C%22east%22%3A-117.22952091247559%2C%22south%22%3A32.988193312083624%2C%22north%22%3A33.00726947554699%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A20544%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sch%22%3A%7B%22value%22%3Atrue%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A14%7D'
solanabeach_params = {
    'searchQueryState': '{"pagination":{},"usersSearchTerm":"Solana Beach, CA","mapBounds":{"west":-117.28874408752442,"east":-117.22952091247559,"south":32.988193312083624,"north":33.00726947554699},"regionSelection":[{"regionId":20544,"regionType":6}],"isMapVisible":true,"filterState":{"sch":{"value":true},"manu":{"value":false},"land":{"value":false}},"isListVisible":true,"mapZoom":14}'
                    }
solanabeach_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'cookie': 'zguid=23|%24bf6750d0-ffc4-4736-bc7c-737656f6257b; _ga=GA1.2.1587369970.1595271965; zjs_anonymous_id=%22bf6750d0-ffc4-4736-bc7c-737656f6257b%22; _pxvid=1053c55a-cabc-11ea-9cc4-0242ac120005; _gcl_au=1.1.944134186.1595271970; _fbp=fb.1.1595271970915.211190209; __gads=ID=e7ff6ce23c96776f:T=1595271971:S=ALNI_MY1JXF--UcnujtoZ-R53SS6jrBTJA; G_ENABLED_IDPS=google; zjs_user_id=%22X1-ZU12ozzrc6awumh_xpn1%22; OptanonConsent=isIABGlobal=false&datestamp=Mon Jul 20 2020 14:22:41 GMT-0500 (Central Daylight Time)&version=5.11.0&landingPath=NotLandingPage&groups=1%3A1%2C3%3A1%2C4%3A0; g_state={"i_p":1596095107745,"i_l":1}; userid=X|3|6f95515a87e0f877%7C5%7CabzX6KUnhr-u9h5VznTfYMD2lNbKaE0_W1xEnVG9dU0%3D; loginmemento=1|a29e259fbede38288d360dedb0645766d8d1d47f1f6e4366006988baa7d78157; ki_r=; optimizelyEndUserId=oeu1596088138433r0.5257272424942192; _pin_unauth=dWlkPU5qZzNNVGMyWm1ZdFpUUTJZaTAwWXpjd0xXRmhPV1V0WW1Kak1XRXdNakZsWW1NMSZycD1kSEoxWlE; ki_s=; ki_t=1596088052742%3B1597949095124%3B1597951429554%3B5%3B224; zgsession=1|86604a77-c96d-4a8a-b78c-095b42efcd76; DoubleClickSession=true; KruxPixel=true; KruxAddition=true; _gid=GA1.2.612872810.1598407954; JSESSIONID=89B940B5E855310E6E0F52F0B99622DB; ZILLOW_SID=1|AAAAAVVbFRIBVVsVEtTcb3iksrkhx1yTFnFnp8o2IZp4XX0ywk1a4aJenH7ifsHeorNmdZcF4oRi2wxG3eH2OEFmUYmr; _derived_epik=dj0yJnU9a2VyVzhRb1hINHlJbnNhbnFkMFREWU51VlRxSlNiV3Qmbj0wbGdONzVaLXIxV01KZ2NVQksyNUFRJm09NyZ0PUFBQUFBRjlGX3Ew; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; _pxff_bsco=1; _gat=1; intercom-session-xby8p85u=VklSMTFoSGN1TEZBS1hTM0hSTGZpa3hwamkwV0tCMWo0NzYrUERMbFFRTWtFaWZEUWE3MDczWmNOdUp1ajZWVC0tZk1xOUVkWjlpWk00WGI5bFkyTm9QZz09--bb7771a4cde88f564d1c46cdf1d3ec4723d8ee74; _px3=b113d0a70641bbdf184a10798880562fbf62cbf8117b76f6e74b4744adfa6b13:XcEhr2Z6dYMIGQoLdpUadNFpEzLZZ+INXBLg0s6XD2POfHZsPBt5KizZLSiaLkWSnDtsGND2ic5fUKWLRN9YRQ==:1000:kc5TTucPjEF8Og7194pwqpyf/Soax84F6hWDePDfrer7dBSBgMdxtlk0mJEHQGtGXNF1fop9jCKMBzP63b79osOEtOniZbhR12JjwaFn7EHkwCzUp+Xx/WUrOzAKJkJ40ntGx5+B/jVI7zgruwIE0HAa0rf7Zq8f91hskJOYRfs=; _uetsid=0d2e531968d37437d5cce675ebb2eb1c; _uetvid=c2047787b1af49647f427b434721efa6; AWSALB=ZEJGc6axIJrZ1hDPD39Mn6iiaRCVCAVpZu+Jm7OUPCbWFotLFIgEOkKSnJyJnB1FkF6+q/gMYoqGXXyM+ajU9Mih+lcxAXmgcweNy5oEXMLSu5haiDRFF8wV9T7+; AWSALBCORS=ZEJGc6axIJrZ1hDPD39Mn6iiaRCVCAVpZu+Jm7OUPCbWFotLFIgEOkKSnJyJnB1FkF6+q/gMYoqGXXyM+ajU9Mih+lcxAXmgcweNy5oEXMLSu5haiDRFF8wV9T7+; search=6|1601015263064%7Crect%3D33.00726947554699%252C-117.22952091247559%252C32.988193312083624%252C-117.28874408752442%26rid%3D20544%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26sort%3Ddays%26z%3D0%26sch%3Dprcemh1u%26type%3Dhouse%252Ccondo%252Capartment_duplex%252Ctownhouse%26pt%3Dpmf%252Cpf%26fs%3D1%26fr%3D0%26mmm%3D1%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%09%0920544%09%09%09%09%09%09',
    'referer': 'https://www.zillow.com/homes/Solana-Beach,-CA_rb/',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
}

solanabeach = ZillowScraper(solanabeach_url,solanabeach_params,solanabeach_headers)
solanabeach.run(3)


## Observations 

In [ ]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import re

In [ ]:
df = pd.read_csv('sandiego_zillow.csv')

In [ ]:
df.head()

In [ ]:
df['city'].value_counts()

In [ ]:
df.info()

# PRE-PROCESSED DATA

In [ ]:
df_copy = df.copy()

In [ ]:
df = df.replace({'\$':'', 
                  ',':'',
                 'bd':'',
                'bds':'',
                 'ba':'',
           'for sale':'',
                 '\+':'',
               'Est.':'',
                 }, regex=True)

df['bedrooms'] = df['bedrooms'].replace('s','', regex=True)
          


In [ ]:
df.head()

In [ ]:
df['bathrooms'].unique()

In [ ]:
df['bathrooms'].value_counts()

In [ ]:
df['bathrooms'] = df['bathrooms'].replace('--',np.nan, regex=True)

In [ ]:
df['bathrooms'].value_counts().sum()

In [ ]:
df['bathrooms'].isna().sum()

In [ ]:
df = df.dropna()
#drop null values

In [ ]:
df['bathrooms'].isna().sum()

In [ ]:
df['floorsize'].unique()

In [ ]:
df['type'].value_counts()

In [ ]:
df['type'] = df['type'].replace({'Home ':'House ','Pre-forclosure':'Auction'}, value=str)

In [ ]:
df['type'].unique()

In [ ]:
df['type'].value_counts()

In [ ]:
df.loc[df['type']=='Coming soon'].head()

In [ ]:
apartments = df.loc[df['type']=='Apartment ']

In [ ]:
apartments

In [ ]:
apartments['floorsize'].value_counts()

# QUESTIONS:
    - how to get rid of all spaces before and after string.

# MODELS

# RESULTS and CONCLUSION

# INSIGHTS and RECOMMENDATIONS